In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from gp_master import *
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def return_mean_std(y_list):
    mean_list = list()
    std_list = list()
    for one_list in y_list:
        mean_list.append(np.mean(one_list))
        std_list.append(np.std(one_list))
    return mean_list, std_list

In [4]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [5]:
def train_logisitc_regression_(mnist):
    def train_logisitc_regression_(params):
        learning_rate = params[0]
        training_epochs = int(params[1])
        batch_size = int(params[2])
        beta = params[3]
        print "\tLearning rate: " + str(learning_rate) + ", training epochs: " + str(training_epochs) + ", batch size: "+ str(batch_size) + ", beta " + str(beta)
        display_step = 10
        # tf Graph Input
        x = tf.placeholder(tf.float32, [None, 784]) 
        y = tf.placeholder(tf.float32, [None, 10])
        # Set model weights
        W = tf.Variable(tf.zeros([784, 10]))
        b = tf.Variable(tf.zeros([10]))
        # Construct model
        pred = tf.nn.softmax(tf.matmul(x, W) + b)
        # Create regulariser
        regularizer = tf.nn.l2_loss(W)
        # Minimize error using cross entropy
        cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1) + beta * regularizer)
        # Gradient Descent
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
        # Initialize the variables
        init = tf.global_variables_initializer()
        # Start training
        with tf.Session() as sess:
            # Run the initializer
            sess.run(init)
            # Training cycle
            for epoch in range(training_epochs):
                avg_cost = 0.
                total_batch = int(mnist.train.num_examples/batch_size)
                # Loop over all batches
                for i in range(total_batch):
                    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
                    # Run optimization op (backprop) and cost op (to get loss value)
                    _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                                  y: batch_ys})
                    # Compute average loss
                    avg_cost += c / total_batch
                # Display logs per epoch step
                #if (epoch+1) % display_step == 0:
                #    print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
            # Test model
            correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
            # Calculate accuracy
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            accuracy_result = accuracy.eval({x: mnist.test.images, y: mnist.test.labels})
            print "\t\t Accuracy: " + str(accuracy_result)
        return 1 - accuracy_result
    return train_logisitc_regression_

In [6]:
loss = train_logisitc_regression_(mnist)

In [90]:
# set parameters
bounds = np.array([[0, 1], [5, 2000], [20, 2000], [0, 1]])
# maxeps specifies how many BOs we want to run
maxeps = 5
# n_iters is number of iterations we want for each run of BO
n_iters = 100
acqui_eva_num = 3
n_pre_samples = 3
y_list = list()
slice_sample_num = 1
coor_sigma = 5 * np.array([0.2,0.2,0.2,0.2,0.2,0.2])
burn_in = 3
input_dimension = 4
mode = 'MAP'
acqui_mode = 'MCMC'
acqui_sample_num = 3

In [ ]:
for i in range(n_iters):
    y_list.append(list())


for j in range(maxeps):
    print ('Running %d episode' % (j + 1))
    xp, yp  = bayesian_optimisation(slice_sample_num, 
                                    coor_sigma, 
                                    burn_in, 
                                    input_dimension,
                                    n_iters=n_iters, 
                                    sample_loss=loss, 
                                    bounds=bounds,
                                    n_pre_samples=n_pre_samples,
                                    acqui_eva_num = acqui_eva_num,
                                    random_search=False,
                                    greater_is_better = False,
                                    mode = mode,
                                    acqui_mode = acqui_mode,
                                    acqui_sample_num = acqui_sample_num)
    for idx in range(n_iters):
        y_list[idx].append(np.min(yp[:idx+n_pre_samples+1]))

In [78]:
mean_list, std_list = return_mean_std(y_list)

In [ ]:
x = [i+1 for i in range(len(y_list))]
plt.errorbar(x, mean_list, yerr = std_list, fmt = '-o')
plt.show()

In [15]:
# set parameters
bounds = np.array([[0, 1], [5, 2000], [20, 2000], [0, 1]])
# maxeps specifies how many BOs we want to run
maxeps = 10
# n_iters is number of iterations we want for each run of BO
n_iters = 100
acqui_eva_num = 3
n_pre_samples = 3
y_list = list()
slice_sample_num = 1
coor_sigma = 5 * np.array([0.2,0.2,0.2,0.2,0.2,0.2])
burn_in = 3
input_dimension = 4
mode = 'OPT'
acqui_mode = 'OPT'
acqui_sample_num = 3

In [ ]:
for i in range(n_iters):
    y_list.append(list())

for j in range(maxeps):
    print ('Running %d episode' % (j + 1))
    xp, yp = bayesian_optimisation(slice_sample_num, 
                                   coor_sigma, 
                                   burn_in, 
                                   input_dimension,
                                   n_iters=n_iters, 
                                   sample_loss=loss, 
                                   bounds=bounds,
                                   n_pre_samples=n_pre_samples,
                                   acqui_eva_num = acqui_eva_num,
                                   random_search=False,
                                   greater_is_better = False,
                                   mode = mode,
                                   acqui_mode = acqui_mode,
                                   acqui_sample_num = acqui_sample_num)
    for idx in range(n_iters):
        y_list[idx].append(np.min(yp[:idx+n_pre_samples+1]))

In [17]:
mean_list, std_list = return_mean_std(y_list)

In [ ]:
x = [i+1 for i in range(len(y_list))]
plt.errorbar(x, mean_list, yerr = std_list, fmt = '-o')
plt.show()